The imports

In [2]:
!pip install pandas
import pandas as pd



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/31/9e/6ebb433de864a6cd45716af52a4d7a8c3c9aaf3a98368e61db9e69e69a9c/pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata
  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for numpy>=1.22.4 from https://files.pythonhosted.org/packages/8e/8b/1c131ab5a94c1086c289c6e1da1d843de9dbd95fe5f5ee6e61904c9518e2/numpy-2.1.3-cp310-cp310-win_amd64.whl.metadata
  Using cached numpy-2.1.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/11/c3/005fcca25ce078d2cc29fd559379817424e94885510568bc1bc53d7d5846/pytz-2024.2-py2.py3-none-any.whl.metadata
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/a6/ab/7e5f53c3b9d14972843a647d8d7a853969a58aecc7559cb3267302c94774/tzda

In [3]:
# Charger le fichier source
persons_df = pd.read_csv("./ir_ben_r.csv")
persons_df['person_id'] = persons_df.index

# Renommer les colonnes pour correspondre aux noms standardisés
persons_df.rename(columns={
    "NUM_ENQ": "person_source_value",
    "ben_nai_ann": "year_of_birth",
    "ben_nai_moi": "month_of_birth",
    "ben_sex_cod": "gender_concept_id"
}, inplace=True)




# Créer une table Location en extrayant les informations de la table Person
location_df = persons_df[['ben_res_dpt', 'ben_res_reg']].drop_duplicates().reset_index(drop=True)
location_df['location_id'] = location_df.index + 1  # Assignation d'un ID unique pour chaque ligne

persons_df = pd.merge(persons_df, location_df, how='left', on=['ben_res_dpt', 'ben_res_reg'])
new_persons_df=persons_df[["person_source_value","year_of_birth","gender_concept_id","month_of_birth","person_id","location_id"]]


location_df.rename(columns={
    "ben_res_reg" : "state" ,
     "ben_res_dpt": "address_1",
}, inplace=True) 
# Sauvegarder le résultat
new_persons_df.to_csv("./Person.csv", index=False) 
location_df.to_csv("./Location.csv", index=False) 

new_persons_df

,person_source_value,year_of_birth,gender_concept_id,month_of_birth,person_id,location_id
0,DPXX:00000000000000001X,1963,2,12,0,1
1,DPXX:000000000000002X,1971,1,2,1,2
2,DPXX:000000000000003X,1962,1,12,2,2
3,DPXX:000000000000004X,1959,2,3,3,3
4,DPXX:000000000000005X,1998,1,4,4,2
5,DPXX:000000000000006X,1958,1,11,5,4
6,DPXX:000000000000007X,1957,2,12,6,1
7,DPXX:000000000000008X,1964,1,11,7,5
8,DPXX:000000000000009X,1988,1,7,8,6
9,DPXX:000000000000010X,1965,2,3,9,7


In [4]:
!pip install sqlite3
import sqlite3

care_sites = pd.read_csv("./t_mcoaae.csv")

care_sites['care_site_id'] = care_sites.index

"""Comme la colonne soc_rai correspond au nom de l'établissement dans le fichier et non à la raison sociale comme indiqué dans la documentation ATHENA, je l'ai donc considérée comme une erreur et j'ai l'utilisé comme le nom de l'établissement."""

care_sites.rename(columns={"eta_num": "care_site_source_value","soc_rai":"care_site_name"}, inplace=True)

# Ajouter des colonnes vides pour care_site_name et location_id
care_sites["location_id"] = None

# Connexion à la base de données SQLite
conn = sqlite3.connect("healthcare.db")  # Nom de la base de données
cursor = conn.cursor()

# Vérifier si la table existe déjà, sinon la créer
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='care_site';")
table_exists = cursor.fetchone()

if not table_exists:
    # Création de la table care_site si elle n'existe pas déjà
    create_table_query = """
    CREATE TABLE care_site (
        care_site_id INTEGER PRIMARY KEY,      
        care_site_name TEXT,                       
        location_id INTEGER,                      
        care_site_source_value TEXT             
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print("Table care_site créée avec succès.")
else:
    print("La table care_site existe déjà.")
    
# Insérer les données dans la table SQLite
care_sites.to_sql("care_site", conn, if_exists="append", index=False)
print("Données insérées avec succès dans la table care_site.")

# Fermer la connexion à la base de données
conn.close()
care_sites

Table care_site créée avec succès.
Données insérées avec succès dans la table care_site.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


,care_site_source_value,care_site_name,care_site_id,location_id
0,750300360,l'Hôpital Privé des Peupliers,0,None
1,750023772,Pharmacie Plaisance,1,None


In [5]:
!pip install pyspark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import monotonically_increasing_id, lit
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Initialize SparkSession
conf = SparkConf().set("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem")
spark = SparkSession.builder.config(conf=conf).getOrCreate()
# Read CSV file into PySpark DataFrame
df1 = spark.read.csv("./ir_act_v.csv", header=True, inferSchema=True)
df2 = spark.read.csv("./ir_spe_v.csv", header=True, inferSchema=True)
# Process the DataFrame
df1 = df1.withColumnRenamed("pfs_act_nat", "specialty_concept_id") \
       .withColumnRenamed("label", "specialty_source_value")

df2 = df2.withColumnRenamed("pfs_spe_cod", "specialty_concept_id") \
       .withColumnRenamed("label", "specialty_source_value")

merged_df = df1.union(df2)
merged_df = merged_df.distinct()
merged_df = merged_df.withColumn("provider_id", monotonically_increasing_id())
merged_df = merged_df.withColumn("provider_id", merged_df["provider_id"].cast("int"))
#I didnt find any information about provider_source_value so i decided to use the provider_id as a provider_source_value 
merged_df = merged_df.withColumn("provider_source_value", merged_df["provider_id"].cast("string"))

#merged_df.write.parquet("./provider.parquet")
merged_df.show()



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pyspark-3.5.3-py2.py3-none-any.whl
  Obtaining dependency information for py4j==0.10.9.7 from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
+--------------------+----------------------+-----------+---------------------+
|specialty_concept_id|specialty_source_value|provider_id|provider_source_value|
+--------------------+----------------------+-----------+---------------------+
|                  50|            Pharmacien|          0|                    0|
|                  26|      Kinésithérapeute|          1|                    1|
|                   1|   Médecin généraliste|          2|                    2|
|                   6|            Radiologue|          3|                    3|
+--------------------+----------------------+----------